# A simple Goddard problem

<img src="goddard.jpg" width=417 height=512>

[Thumbnail](goddard.jpg)

This well-known problem (see for instance [1],[2]) models the ascent of a rocket through the atmosphere, and we restrict here ourselves to  vertical (monodimensional) trajectories.
The state variables are the altitude, speed and mass of the rocket during the flight, for a total dimension of 3. The rocket is subject to gravity, thrust and drag forces. The final time is free, and the objective is to reach a certain altitude with a minimal fuel consumption, ie a maximal final mass. All units are renormalized.

$$
\left \lbrace
\begin{array}{l}
Max\ m(T)\\
\dot r = v\\
\dot v = -\frac{1}{r^2} + \frac{1}{m} (T_{max} u - D(r,v))\\
\dot m = - b u\\
u(\cdot) \in [0,1]\\
r(0)=1,\ v(0)=0,\ m(0)=1\\
r(1) = 1.01\\
D(r(\cdot),v(\cdot)) \le C\\
T\ is\ free
\end{array}
\right .
$$

The drag term is a function of speed and altitude defined as 
$D(r,v)=Av^2\rho(r)$,
with the volumic mass given by the approximate model 
$\rho(r)=e^{-k(r-r_0)}$.

In the following we use the parameters 
$b=7,\ T_{max}=3.5,\ A=310,\ k=500,\ r_0=1$.

The Hamiltonian is an affine function of the control, so singular arcs may occur. We consider here a path constraint limiting the value of the drag effect: $D(r,v)\le C$. We can observe that depending on the value of C, the control structure changes. In the unconstrained case, the optimal trajectory presents a singular arc with a non-maximal thrust. When C is set under the maximal value attained by the drag in the unconstrained case, a constrained arc appears. If C is small enough, the singular arc is completely replaced by the constrained arc.

In [1]:
!pygmentize problem.cpp

// +++DRAFT+++ This class implements the OCP functions
// It derives from the generic class bocop3OCPBase
// OCP functions are defined with templates since they will be called
// from both the NLP solver (double arguments) and AD tool (ad_double arguments)
//#pragma once

#include <OCP.h>
// ///////////////////////////////////////////////////////////////////


// aux functions

// FUNCTION FOR GODDARD DRAG
// drag = 310 v^2 exp (-500(r-1))
// Arguments:
// r: radius
// v: velocity
template <typename Variable>
inline Variable drag(const Variable r, const Variable v, double A, double k, double r0)
{
  Variable drag = A * v * v * exp(-k*(fabs(r)-r0));
  return drag;
}

// FUNCTION FOR GRAVITY
// g = 1 / r^2
// Arguments:
// r: radius
template <typename Variable>
inline Variable grav(const Variable r)
{
  Variable grav = 1e0 / r / r;
  return grav;
}

// FUNCTION FOR THRUST (GODDARD)
// Variable = u * Tmax
// Arguments:
// r: radius
template <typename Variable>
inline Variable thrust(const

In [1]:
%matplotlib inline
import bocop

In [2]:
# build problem in current folder
bocop.build(cmake_options='-DCMAKE_CXX_COMPILER=g++')

[EXEC] > ['cmake -DCMAKE_BUILD_TYPE=Debug -DPROBLEM_DIR=/home/martinon/bocop/bocop3/bocop/examples/goddard -DCPP_FILE=problem.cpp -DCMAKE_CXX_COMPILER=g++ /home/martinon/bocop/bocop3/bocop']
>	-- The C compiler identification is GNU 9.3.0
>	-- The CXX compiler identification is GNU 9.3.0
>	-- Detecting C compiler ABI info
>	-- Detecting C compiler ABI info - done
>	-- Check for working C compiler: /home/martinon/miniconda3/envs/bocop-dev/bin/x86_64-conda-linux-gnu-cc - skipped
>	-- Detecting C compile features
>	-- Detecting C compile features - done
>	-- Detecting CXX compiler ABI info
>	-- Detecting CXX compiler ABI info - done
>	-- Check for working CXX compiler: /usr/bin/g++ - skipped
>	-- Detecting CXX compile features
>	-- Detecting CXX compile features - done
>	-- Problem path: /home/martinon/bocop/bocop3/bocop/examples/goddard
>	-- Using CPPAD found at /home/martinon/miniconda3/envs/bocop-dev/include/cppad/..
>	-- Using IPOPT found at /home/martinon/miniconda3/envs/bocop-dev/li

0

In [3]:
# execute with graphical display of iterations
bocop.run(graph=1)

interactive(children=(IntSlider(value=20, continuous_update=False, description='iteration', max=20), Output())…

Done


In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

solution = bocop.readSolution()
bocop.low_diagnose(solution)    

Loading solution:  problem.sol


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
print("Bocop returns status {} with objective {:2.4g} and constraint violation {:2.4g}".format(solution.status,solution.objective,solution.constraints))
p0 = []
for i in range(solution.dim_state):
    p0.append(solution.costate[i][0])
print("Costate at first time stage (t0+h/2): ",p0)
print("Multipliers for initial conditions: ",solution.boundarycond_multipliers[0:solution.dim_state])

Bocop returns status 0 with objective -0.6298 and constraint violation 2.185e-13
Costate at first time stage (t0+h/2):  [53.0586254065787, 1.81248579905455, 0.689123980123291]
Multipliers for initial conditions:  [53.06865501  1.90672564  0.67678818]
